In [54]:
using System;
using System.Collections.Generic;
using System.Linq;

//Определение перечисления для статусов заказа
public enum OrderStatus
{
    Создан,
    В_процессе,
    Выполнен
}

public class Employee
{
    public string FirstName { get; }
    public string LastName { get; }
    public string Position { get; }
    public int ProcessedOrdersCount { get; private set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        ProcessedOrdersCount = 0; //Инициализация счетчика обработанных заказов
    }

    public void IncrementProcessedOrders()
    {
        ProcessedOrdersCount++;
    }

    // Метод для обработки уведомлений
    public void Notify(string message)
    {
        Console.WriteLine($"Уведомление для {FirstName} {LastName}: {message}");
    }
}

public class Customer
{
    public string FirstName { get; }
    public string LastName { get; }
    public string Phone { get; }

    public Customer(string firstName, string lastName, string phone)
    {
        FirstName = firstName;
        LastName = lastName;
        Phone = phone;
    }
}

public class Order
{
    private static int _orderCounter = 0; //Счетчик заказов
    public int OrderId { get; }
    public string Description { get; }
    public Customer Customer { get; }
    public OrderStatus Status { get; set; }
    public Employee AssignedEmployee { get; set; }

    public Order(string description, Customer customer)
    {
        OrderId = ++_orderCounter;
        Description = description;
        Customer = customer;
        Status = OrderStatus.Создан;
    }

    public void GetDeliveryDetails()
    {
        Console.WriteLine($"Заказ #{OrderId}: {Description}, Заказчик: {Customer.FirstName} {Customer.LastName}, Статус: {Status}");
    }
}

public class Company
{
    public string Name { get; }
    private List<Employee> Employees { get; } = new List<Employee>();
    private List<Order> Orders { get; } = new List<Order>();

    public Company(string name)
    {
        Name = name;
    }

    public void AddEmployees(List<Employee> employees)
    {
        Employees.AddRange(employees); //Добавление сотрудников в список
    }

    public void AddOrders(List<Order> orders)
    {
        Orders.AddRange(orders);
    }

    public void AssignEmployeeToOrder(int orderId, Employee employee)
    {
        var order = Orders.FirstOrDefault(o => o.OrderId == orderId); //Поиск заказа по ID
        if (order != null)
        {
            order.AssignedEmployee = employee;
            Console.WriteLine($"Сотрудник {employee.FirstName} {employee.LastName} назначен на заказ #{orderId}.");
        }
        else
        {
            Console.WriteLine("Заказ не найден.");
        }
    }

    public void ChangeOrderStatus(int orderId, OrderStatus status)
    {
        var order = Orders.FirstOrDefault(o => o.OrderId == orderId);
        if (order != null)
        {
            order.Status = status;

            //Уведомляем только назначенного сотрудника об изменении статуса заказа
            if (order.AssignedEmployee != null)
            {
                order.AssignedEmployee.Notify($"Статус заказа #{orderId} изменен на {status}.");
                order.AssignedEmployee.IncrementProcessedOrders();
            }
        }
    }

    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        return Orders.Where(order => order.Status == status).ToList(); //Возврат списка заказов с указанным статусом

    }

    public List<Order> GetOrdersByCustomer(Customer customer)
    {
        return Orders.Where(order => order.Customer == customer).ToList();
    }

    public void PrintEmployeeReport()
    {
        Console.WriteLine("Отчет по сотрудникам:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"{employee.FirstName} {employee.LastName} - {employee.Position}, Обработано заказов: {employee.ProcessedOrdersCount}");
        }
    }


    public void DisplayEmployees()
    {
        Console.WriteLine($"Сотрудники компании {Name}:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"{employee.FirstName} {employee.LastName}, Должность: {employee.Position}");
        }
    }
}


//Делегат для уведомления сотрудников
public delegate void OrderStatusNotificationHandler(string message);

public delegate void DisplayDelegate();

//Работа с List для компаний
List<Company> companyList = new List<Company>();

//Создаем компанию
Company company = new Company("UpTIME");
companyList.Add(company); // Добавляем компанию в список

//Создаем списки сотрудников и заказчиков
List<Employee> employees = new List<Employee>
{
    new Employee("Анастасия", "Косишнева", "Менеджер"),
    new Employee("Валерия", "Шананина", "Разработчик")
};

List<Customer> customers = new List<Customer>
{
    new Customer("Аделия", "Шаукерова", "7-912-396-58-86"),
    new Customer("Егор", "Полянцев", "7-919-942-29-18"),
    new Customer("Егор", "Полянцев", "7-919-942-29-18")
};

company.AddEmployees(employees);

Console.WriteLine("Список сотрудников компании:");
company.DisplayEmployees();
Console.WriteLine();

//Добавляем заказы с указанием заказчиков
company.AddOrders(new List<Order>
{
    new Order("Заказ на разработку сайта", customers[0]),
    new Order("Заказ на создание мобильного приложения", customers[1]),
    new Order("Заказ на создание десктопного приложения", customers[1])


});

//Назначаем делегату метод GetDeliveryDetails
DisplayDelegate displayOption1 = company.GetOrdersByStatus(OrderStatus.Создан)[0].GetDeliveryDetails;
DisplayDelegate displayOption2 = company.GetOrdersByStatus(OrderStatus.Создан)[1].GetDeliveryDetails;
DisplayDelegate displayOption3 = company.GetOrdersByStatus(OrderStatus.Создан)[2].GetDeliveryDetails;

Console.WriteLine("Делегаты:");
// Вызов методов через делегаты
displayOption1();
displayOption2();
displayOption3();
Console.WriteLine();


Console.WriteLine("События:");
// Назначаем сотрудников на заказы
company.AssignEmployeeToOrder(1, employees[0]);
company.AssignEmployeeToOrder(2, employees[1]);
company.AssignEmployeeToOrder(3, employees[1]);

Console.WriteLine();

// Изменяем статус заказа и выводим событие
company.ChangeOrderStatus(1, OrderStatus.В_процессе);
company.ChangeOrderStatus(2, OrderStatus.Выполнен);
company.ChangeOrderStatus(3, OrderStatus.В_процессе);
Console.WriteLine();

// Получаем и выводим все завершенные заказы
var ВыполненOrders = company.GetOrdersByStatus(OrderStatus.Выполнен);
        
Console.WriteLine("Завершенные заказы:");
foreach (var order in ВыполненOrders)
{
    Console.WriteLine($"Заказ #{order.OrderId}: {order.Description}, Заказчик: {order.Customer.FirstName} {order.Customer.LastName}, Статус: {order.Status}");
}
Console.WriteLine();

// Поиск заказов по заказчику
var ordersByCustomer = company.GetOrdersByCustomer(customers[1]);
        
Console.WriteLine($"Заказы для заказчика {customers[1].FirstName} {customers[1].LastName}:");
foreach (var order in ordersByCustomer)
{
    Console.WriteLine($"Заказ #{order.OrderId}: {order.Description}, Статус: {order.Status}");
}
Console.WriteLine();

// Печатаем отчет по сотрудникам
company.PrintEmployeeReport();


Список сотрудников компании:
Сотрудники компании UpTIME:
Анастасия Косишнева, Должность: Менеджер
Валерия Шананина, Должность: Разработчик

Делегаты:
Заказ #1: Заказ на разработку сайта, Заказчик: Аделия Шаукерова, Статус: Создан
Заказ #2: Заказ на создание мобильного приложения, Заказчик: Егор Полянцев, Статус: Создан
Заказ #3: Заказ на создание десктопного приложения, Заказчик: Егор Полянцев, Статус: Создан

События:
Сотрудник Анастасия Косишнева назначен на заказ #1.
Сотрудник Валерия Шананина назначен на заказ #2.
Сотрудник Валерия Шананина назначен на заказ #3.

Уведомление для Анастасия Косишнева: Статус заказа #1 изменен на В_процессе.
Уведомление для Валерия Шананина: Статус заказа #2 изменен на Выполнен.
Уведомление для Валерия Шананина: Статус заказа #3 изменен на В_процессе.

Завершенные заказы:
Заказ #2: Заказ на создание мобильного приложения, Заказчик: Егор Полянцев, Статус: Выполнен

Заказы для заказчика Егор Полянцев:
Заказ #2: Заказ на создание мобильного приложения, С